In [157]:
import os,shutil,pickle,tqdm,sys,random,re,string,pause, datetime,glob
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# # The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import keras
import sent2vec
import seq2seq
from seq2seq.models import AttentionSeq2Seq
from seq2seq.models import Seq2Seq

import tensorflow as tf
import tensorboard as tb

import numpy as np
import pandas as pd
import tqdm
from tqdm import tqdm
from random import shuffle
from math import log, floor

from keras.utils import multi_gpu_model

# from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
from keras.metrics import *

import gensim
from gensim.models.word2vec import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *

from sklearn.model_selection import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn.metrics.pairwise import *
from sklearn.neighbors import KNeighborsClassifier
# from collections import Counter
from keras.utils.generic_utils import *
from keras import regularizers
import unicodedata as udata
from keras.applications import *
from keras.preprocessing.image import *

from keras import backend 
from imblearn.ensemble import *
from imblearn.combine import *
# from python.keras import backend 
# Embedding(10,20)
from keras_transformer.extras import ReusableEmbedding, TiedOutputEmbedding
from keras_transformer.position import TransformerCoordinateEmbedding
from keras_transformer.transformer import TransformerACT, TransformerBlock
from keras_transformer.bert import (
    BatchGeneratorForBERT, masked_perplexity,
    MaskedPenalizedSparseCategoricalCrossentropy)

import keras_metrics as km
from keras_trans_mask import RemoveMask, RestoreMask

from keras_multi_head import *
from keras_self_attention import SeqSelfAttention

from sklearn.metrics.pairwise import *
from sklearn.model_selection import *
from sklearn.manifold import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn import preprocessing

In [18]:
from models import transformer_bert_model
from bpe import BPEEncoder
from utils import *

In [19]:
root_dir = './data/tree-rep-profiles_o2o/'
train_pid_paths = pickle.load(open(root_dir + 'train_only_pid_paths.pkl','rb'))
valid_pid_paths = pickle.load(open(root_dir + 'valid_pid_paths.pkl','rb'))
test_pid_paths = pickle.load(open(root_dir + 'test_pid_paths.pkl','rb'))
exp_pid_paths = pickle.load(open(root_dir + 'exp_pid_paths.pkl','rb'))

train_emb_api,train_emb , train_rep_ans = pickle.load(open(root_dir + 'pids_train_only.pkl','rb'))
valid_emb_api,valid_emb, valid_rep_ans = pickle.load(open(root_dir + 'pids_valid.pkl','rb'))
test_emb_api,test_emb ,test_rep_ans = pickle.load(open(root_dir + 'pids_test.pkl','rb'))
exp_api,exp_emb = pickle.load(open(root_dir + 'pids_exp.pkl','rb'))
emb_matrix = pickle.load(open('data/api_emb_matrix.pkl','rb'))

In [107]:
def basic_statistics(all_length):
    '''
    input: length list of elements
    output1: mean、std、mode、min、q1、median(q2)、q3、max、iqr、outlier、far out
    output2: statistics graph、10%~90% form
    '''
    stat_dict = {}
    stat_dict['mean'] = np.mean(all_length)
    stat_dict['std'] = np.std(all_length)
    stat_dict['mode'] = np.argmax(np.bincount(all_length))
    stat_dict['min'] = np.min(all_length)
    stat_dict['q1'] = np.quantile(all_length,0.25)
    stat_dict['median'] = np.quantile(all_length,0.5)
    stat_dict['q3'] = np.quantile(all_length,0.75)
    stat_dict['max'] = np.max(all_length)
    stat_dict['iqr'] = stat_dict['q3'] - stat_dict['q1']
    stat_dict['outlier'] = stat_dict['q3'] + 1.5*stat_dict['iqr']
    stat_dict['far_out'] = stat_dict['q3'] + 3*stat_dict['iqr']
    for i in [10,20,30,40,50,60,70,80,90,100]:
        stat_dict[str(i)+'%'] = np.percentile(all_length,i)
    return pd.DataFrame.from_dict(stat_dict,orient='index',columns=['length'])

model arch與參數設置要與前面一致

In [20]:
def bin_acc(y_true, y_pred):
    return binary_accuracy(y_true, y_pred)

In [21]:
model_save_path = './model/o2o_stage_gru_selfatt/byterep_2ndStage_0706_gruatt_sent2vec.h5' #改
model = load_model(model_save_path+'_all.h5',custom_objects={'bin_acc': bin_acc, 'SeqSelfAttention': SeqSelfAttention})
model.load_weights(model_save_path)
model.summary()
thr = 0.482

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sent_ids (InputLayer)           (None, 226)          0                                            
__________________________________________________________________________________________________
sent_emb (InputLayer)           (None, 226, 768)     0                                            
__________________________________________________________________________________________________
masking_10 (Masking)            (None, 226)          0           sent_ids[0][0]                   
__________________________________________________________________________________________________
masking_9 (Masking)             (None, 226, 768)     0           sent_emb[0][0]                   
__________________________________________________________________________________________________
api_emb (E

In [31]:
emb_model = Model(inputs=model.inputs,  outputs=model.layers[5].output)
emb_model.load_weights(model_save_path,by_name=True)
emb_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sent_ids (InputLayer)           (None, 226)          0                                            
__________________________________________________________________________________________________
sent_emb (InputLayer)           (None, 226, 768)     0                                            
__________________________________________________________________________________________________
masking_10 (Masking)            (None, 226)          0           sent_ids[0][0]                   
__________________________________________________________________________________________________
masking_9 (Masking)             (None, 226, 768)     0           sent_emb[0][0]                   
__________________________________________________________________________________________________
api_emb (E

## Train
* avg self-vector only in DEV existed's trees
* create important call seq of train data in profile_avg
* No ind REP#: 979
* 4104it [10:44,  6.18it/s]

In [202]:


def avg_train_vector(root_dir,thr):
    no_ind_rep = 0
    tree_vec = []
    if not os.path.exists(root_dir):
        os.makedirs(root_dir,exist_ok=True)
    for path,api,emb in (zip(train_pid_paths,train_emb_api,train_emb)): #tqdm
        recent_fam = path.split('/')[-3].split('_')[0]
        recent_tree = path.split('/')[-2]
        hash_id = path.split('/')[-1][:5]
        pid_id = path.split('/')[-1].split('_')[-1].split('.')[0][-3:]
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if length < 11:
            print('No common rep. Should not in a tree:',path)
            continue
        assert api[:length][-1] != 0
        assert api[:length+1][-1] == 0
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0)
        final_emb_e = emb_model.predict([api_e,emb_e]) #(1,length,768)
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e])
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr)
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if (sum(emb) <0) or (sum(emb)>0):
                rep_emb.append(emb)
        rep_emb_avg = np.average(rep_emb,axis=0) #把自己各api的embedding取平均，應該一定要有，不然就代表這個profile不該屬於這棵tree 
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):
                no_ind_rep+=1
                continue #沒有individual REP
        except ValueError: #正常profile
            pass
        rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)
        pickle.dump(file=open(root_dir+recent_fam+'_'+recent_tree+'_'+hash_id+'_'+pid_id+'.pkl','wb'),obj=rep_emb_avg)
#     print('No ind REP#:',no_ind_rep)
    return no_ind_rep

In [ ]:
no_ind_rep_train_li = []
for i in tqdm(range(1,100)): #system: 500 (0.002)
    thr = i/100
    root_dir = './data/tree-rep-profiles_o2o/profile_avg_exp/'+str(thr)+'/' # data set dir
    no_ind_rep = avg_train_vector(root_dir,thr)
    no_ind_rep_train_li.append(no_ind_rep)


  0%|          | 0/99 [00:00<?, ?it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

 12%|█▏        | 12/99 [2:13:43<16:09:48, 668.83s/it]

## DEV/Test
* cos-sim of each individual rep
* TopN (cos-sim>0.8)
* 每個pid歸類到的tree_name一樣就成功

In [119]:
ind_rep_dir = './data/tree-rep-profiles_o2o/profile_avg/' #要有最後的反斜線
ind_reps = next(os.walk(ind_rep_dir))[2]
ind_rep_tree = []
ind_rep_fam = []
for i,c in enumerate(ind_reps):
    ind_rep = pickle.load(open(ind_rep_dir + c,'rb')) #每個tree的centroid pickle
    if i==0:
        ind_reps_matrix = ind_rep # 1st time
    else:
        ind_reps_matrix = np.concatenate((ind_reps_matrix,ind_rep),axis=0) #依順序合在一起 
    ind_rep_tree.append(c.split('_')[1])
    ind_rep_fam.append(c.split('_')[0].split('.')[-1])
assert ind_reps_matrix.shape[0] == len(ind_rep_tree) == len(ind_rep_fam)

In [102]:
def EM_index(score):
    '''
    input: cosin sim 分數
    output: 同分的index
    '''
    scores = score[0].argsort() #相似度小到大
    scores = np.flip(scores) # 相似度大到小
    tmp_score = np.around(np.max(score),decimals=4) # 取道小數點地五為四捨五入到第四位
    em_id = []
    for s in scores:
        now_score = np.round(score[0][s],decimals=4)
        if now_score ==tmp_score:
            em_id.append(s)
    return em_id

In [115]:
def inf_evaluate(inf_pid_paths,inf_emb_api,inf_emb,ind_reps_matrix=ind_reps_matrix,ind_rep_tree=ind_rep_tree):
    em_up = 0
    no_ind_rep = 0
    bound_accuracy = []
    wrong_tree = {}
    for path,api,emb in tqdm(zip(inf_pid_paths,inf_emb_api,inf_emb)):
        recent_tree = path.split('/')[-2]
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if length < 1:
            print('Empty profile:',path)
            continue
        assert api[:length][-1] != 0 #確認profile跟所load 近來的vector是一致的
        assert api[:length+1][-1] == 0
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0)
        final_emb_e = emb_model.predict([api_e,emb_e]) #(1,length,768)
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e])
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr)
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if (sum(emb) <0) or (sum(emb)>0):
                rep_emb.append(emb)
        rep_emb_avg = np.average(rep_emb,axis=0) #把自己各api的embedding取平均，應該一定要有，不然就代表這個profile不該屬於這棵tree 
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):
                no_ind_rep+=1
                continue #沒有individual REP
        except ValueError: #正常profile
            pass
        rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)
        score = cosine_similarity(rep_emb_avg,ind_reps_matrix)
        
        scores = score[0].argsort() #相似度小到大
        scores = np.flip(scores) # 相似度大到小
        for idx in scores:
            if ind_rep_tree[idx] == recent_tree:
                bound_accuracy.append(score[0][idx]) #最差都是幾分的時候合併
        
        idx_list = EM_index(score) # exact match的id
        em_tree = []
        for idx in idx_list:
            em_tree.append(ind_rep_tree[idx])
        em_tree = list(set(em_tree)) # exact match的tree name
        if recent_tree in em_tree:
            em_up = em_up + 1
        else:
            wrong_tree[path]=em_tree
    down = len(inf_pid_paths)
    print('ExactMatchAccuracyScore:',em_up/down,'EM#:/all#:/NoIndREP#:',em_up,down,no_ind_rep,
         'ExactMatchPrecision(w/o No ind REP):',em_up/(down-no_ind_rep)
         )
    return bound_accuracy,wrong_tree #真正正確的tree是在幾%相似度才有? 、{[正確 path]=predict list}

In [116]:
bound_accuracy,wrong_tre = inf_evaluate(test_pid_paths,test_emb_api,test_emb)
basic_statistics(bound_accuracy)

5it [00:00,  5.89it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
11it [00:01,  6.50it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
29it [00:04,  6.25it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret 

376it [00:56,  6.57it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
385it [00:58,  6.75it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
393it [00:59,  6.55it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(

Exact Match Accuracy Score: 0.734251968503937 EM#:/all#:/No ind REP#: 373 508 125 Exact Match Accuracy Score (w/o No ind REP): 0.9738903394255874


In [112]:
basic_statistics(bound_accuracy)

,length
mean,9.948047e-01
std,1.627907e-02
mode,0.000000e+00
min,3.652893e-01
q1,1.000000e+00
median,1.000000e+00
q3,1.000000e+00
max,1.000000e+00
iqr,1.332268e-15
outlier,1.000000e+00


In [55]:
Exact Match Accuracy Score: 0.734251968503937

'G395'

In [95]:
from numpy.linalg import inv
score = cosine_similarity(ind_rep,ind_reps_matrix)
def EM_index(score):
    '''
    input: cosin sim 分數
    output: 同分的index
    '''
    scores = score[0].argsort() #相似度小到大
    scores = np.flip(scores) # 相似度大到小
    tmp_score = np.around(np.max(score),decimals=4) #
    em_id = []
    for s in scores:
        now_score = np.round(score[0][s],decimals=4)
        if now_score ==tmp_score:
            em_id.append(s)
    return em_id
#     print(score[0][s])
#     break
print(score)
# scores = np.expand_dims(scores,axis=0)


[[0.9714486  0.93313132 0.93886226 ... 0.97578548 0.98307781 1.        ]]


## Loner EXP
* 同分遞移
* 同hash、不同pid歸類fam

In [125]:
def statistic_hash_pids(root_dir):
    fam_dir = next(os.walk(root_dir))[1]
    all_pids= []
    all_pid_list = []
    tree_count = 0
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir + fam))[1]
        tree_count += len(tree_dir)
        for tree in tree_dir:
            in_directory = root_dir + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            hash_list = [f.split('/')[-1].split('_')[0] for f in hl_list]
            pid_list = [f.split('/')[-1] for f in hl_list]
            all_pids.extend(hash_list)
            all_pid_list.extend(pid_list)
#             print(fam,len(hash_list)) #DEBUG
    all_hash = set(all_pids)
    print('Families#:',len(fam_dir),'Samples#:',len(all_hash),',Processes#:',len(set(all_pid_list)),',Trees#:',tree_count)
    return all_hash, list(set(all_pid_list))

In [137]:
temp = statistic_hash_pids('./data/tree-rep-profiles_o2o/EXP/')



100%|██████████| 45/45 [00:00<00:00, 1513.82it/s]

Families#: 45 Samples#: 3328 ,Processes#: 3893 ,Trees#: 45


In [154]:
def topN_index(score,sim_score=0.9): #改:多大相似度以上就拿進來
    '''
    input: cosin sim 分數
    output: topN的index
    '''
    scores = score[0].argsort() #相似度小到大
    scores = np.flip(scores) # 相似度大到小
    tmp_score = np.around(np.max(score),decimals=4) # 取道小數點地五為四捨五入到第四位
    topN_id = []
    for idx in scores:
        if score[0][idx] < sim_score:
            break
        topN_id.append(idx)
    return topN_id

In [155]:

def loner_evaluate(inf_pid_paths,inf_emb_api,inf_emb,ind_reps_matrix=ind_reps_matrix,ind_rep_fam=ind_rep_fam):
    em_up = 0
    no_ind_rep_hash = []
    bound_accuracy = []
    wrong_fam = {}
    processed_hash = {}
    processed_hash_topN = []
    i=-1
    topN_up = 0
    for path,api,emb in tqdm(zip(inf_pid_paths,inf_emb_api,inf_emb)):
        i+=1
        recent_fam = path.split('/')[-3] #EX: fakeav_fakealert
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if length < 2:
            print('Empty profile:',path)
            continue
        try:
            assert api[:length][-1] != 0 #確認profile跟所load 近來的vector是一致的
            assert api[:length+1][-1] == 0
        except:
            print('===Assertion ERR:',i,path)
            continue
        recent_hash = path.split('/')[-1].split('_')[0]
        if recent_hash in processed_hash.keys():
            if processed_hash[recent_hash] == 1:
                continue
        else:
            processed_hash[recent_hash] = 0        
        
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0)
        final_emb_e = emb_model.predict([api_e,emb_e]) #(1,length,768)
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e])
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr)
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if (sum(emb) <0) or (sum(emb)>0):
                rep_emb.append(emb)
        rep_emb_avg = np.average(rep_emb,axis=0) #把自己各api的embedding取平均，應該一定要有，不然就代表這個profile不該屬於這棵tree 
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):
                no_ind_rep_hash.append(recent_hash)
                continue #沒有individual REP
        except ValueError: #正常profile
            pass
        rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)
        score = cosine_similarity(rep_emb_avg,ind_reps_matrix)
        
        scores = score[0].argsort() #相似度小到大
        scores = np.flip(scores) # 相似度大到小
        for idx in scores:
            if ind_rep_fam[idx] == recent_fam:
                bound_accuracy.append(score[0][idx]) #最差都是幾分的時候合併 #by_process
        
        idx_list = EM_index(score) # exact match的id
        idx_list_topN = topN_index(score)
        em_fam = []
        for idx in idx_list:
            em_fam.append(ind_rep_fam[idx])
        em_fam = list(set(em_fam)) # exact match的tree name
        topN_fam = []
        for idx in idx_list_topN:
            topN_fam.append(ind_rep_fam[idx])
        topN_fam = list(set(topN_fam))
        flag = False #預設玫分對類別
        for fam in em_fam:
            if fam in recent_fam:
                processed_hash[recent_hash] = 1
                flag = True #分對類
                break
        if not flag: #依然沒分對類別
            wrong_fam[path]=em_fam #by_process
        if recent_hash not in processed_hash_topN:
            for fam in topN_fam:
                if fam in recent_fam:
                    topN_up = topN_up + 1
                    break
            processed_hash_topN.append(recent_hash)
    em_up = sum(list(processed_hash.values()))
    down = len(processed_hash)
    no_ind_rep = len(list(set(no_ind_rep_hash) - set(processed_hash.keys())))#number
    print('ExactMatchAccuracyScore:',em_up/down,'EM_hash#:/all_hash#:/NoIndREP_sample#:',em_up,down,no_ind_rep,
         'ExactMatchPrecision(w/o No ind REP):',em_up/(down-no_ind_rep),
          'TopNAccScore:',topN_up/len(processed_hash_topN),'TopNAccScore(w/o No ind REP):',topN_up/(len(processed_hash_topN)-no_ind_rep)
         )
    return bound_accuracy,wrong_fam,processed_hash #真正正確的tree是在幾%相似度才有? 、{[正確 path]=predict list}

In [156]:
temp = loner_evaluate(exp_pid_paths,exp_api,exp_emb)


0it [00:00, ?it/s]
1it [00:00,  3.97it/s]
2it [00:00,  4.52it/s]
3it [00:00,  5.05it/s]
5it [00:00,  6.26it/s]

Empty profile: ./data/tree-rep-profiles_o2o/EXP/parite/G0/e86fcc08eb8dec19fdac33f2b21e8a55bdf677c03d51fc438373187788c477e4_3748.profile


/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

7it [00:00,  6.64it/s]
8it [00:01,  6.51it/s]
9it [00:01,  6.46it/s]
11it [00:01,  7.57it/s]

Empty profile: ./data/tree-rep-profiles_o2o/EXP/parite/G0/bab4f83751a14be7ba6d757674024de4a2c043556a51039eb4708e8b5de0683d_2892.profile
Empty profile: ./data/tree-rep-profiles_o2o/EXP/parite/G0/67df737fad5c83781f20c9485749ef999f88e012c1ecdadabf1c8ab3b8f14a00_432.profile



13it [00:01,  8.50it/s]
15it [00:01,  9.29it/s]

...

Empty profile: ./data/tree-rep-profiles_o2o/EXP/virut/G0/c68c2c0d0d6c9788293c8d22891961fb802887fa7fa06b2ee83e6b8d1737e5af_3332.profile



3860it [08:38,  7.91it/s]
3862it [08:39,  8.94it/s]

Empty profile: ./data/tree-rep-profiles_o2o/EXP/virut/G0/142f7bedf10ef234b62fbf9a20f95e0c19e00c9a4a6f983c397515a18c999abf_3120.profile



3863it [08:39,  8.29it/s]
3864it [08:39,  7.36it/s]
3865it [08:39,  7.36it/s]
3866it [08:39,  6.95it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

3867it [08:39,  7.52it/s]
3868it [08:39,  6.80it/s]
3869it [08:40,  6.91it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

3871it [08:40,  7.17it/s]
3872it [08:40,  6.87it/s]
3873it [08:40,  6.67it/s]
3874it [08:40,  6.73it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: Runt

ExactMatchAccuracyScore: 0.07257799671592775 EM_hash#:/all_hash#:/NoIndREP_sample#: 221 3045 0 ExactMatchPrecision(w/o No ind REP): 0.07257799671592775 TopNAccScore: 0.9130901287553648 TopNAccScore(w/o No ind REP): 0.9130901287553648


ExactMatchAccuracyScore: 0.07257799671592775 EM_hash#:/all_hash#:/NoIndREP_sample#: 221 3045 0 ExactMatchPrecision(w/o No ind REP): 0.07257799671592775 TopNAccScore: 0.9477825464949928 TopNAccScore(w/o No ind REP): 0.9477825464949928

(0.8)

In [146]:
basic_statistics(temp[0])

,length
mean,0.934728
std,0.078276
mode,0.000000
min,0.257708
q1,0.919924
median,0.955627
q3,0.975992
max,1.000000
iqr,0.056068
outlier,1.060095


## Loner kNN
* align with loner dir encode

In [161]:
loner_dir = './data/tree-rep-profiles_o2o/EXP/'
fam_exp = next(os.walk(loner_dir))[1]
encode_fam = {}
for i,v in enumerate(fam_exp):
    encode_fam[v] = i
pickle.dump(file=open('data/tree-rep-profiles_o2o/encode_fam.pkl','wb'),obj=encode_fam)
encode_fam

{'parite': 0,
 'ipamor': 1,
 'yantai': 2,
 'outbrowse': 3,
 'megasearch': 4,
 'autoit': 5,
 'hotbar': 6,
 'lydra': 7,
 'directdow': 8,
 'solimba': 9,
 'domaiq': 10,
 'eggnog': 11,
 'zusy': 12,
 'picsys': 13,
 'sirefef': 14,
 'browsefox': 15,
 'zbot': 16,
 'ursu': 17,
 'kazy': 18,
 'sytro': 19,
 'pcclient': 20,
 'shodi': 21,
 'allaple': 22,
 'graftor': 23,
 'katusha': 24,
 'mira': 25,
 'fesber': 26,
 'razy': 27,
 'morstar': 28,
 'expiro': 29,
 'fakeav_fakealert': 30,
 'sality': 31,
 'symmi': 32,
 'softpulse': 33,
 'vilsel': 34,
 'bdmj': 35,
 'ramnit': 36,
 'installerex': 37,
 'vobfus': 38,
 'somoto': 39,
 'clickdownload': 40,
 'downloa_soft': 41,
 'mplug': 42,
 'medfos': 43,
 'virut': 44}

In [165]:
# train data prepare
ind_rep_dir = './data/tree-rep-profiles_o2o/profile_avg/' #要有最後的反斜線
ind_reps = next(os.walk(ind_rep_dir))[2]
kNN_train_Y = []
for i,c in enumerate(ind_reps):
    ind_rep = pickle.load(open(ind_rep_dir + c,'rb')) #每個tree的centroid pickle
    fam_name = c.split('_')[0].split('.')[-1]
    for j,v in enumerate(list(encode_fam.keys())):
        if fam_name in v:
            kNN_train_Y.append(j)
            break
    if i==0:
        kNN_train_X = ind_rep # 1st time
    else:
        kNN_train_X = np.concatenate((kNN_train_X,ind_rep),axis=0) #依順序合在一起 
assert kNN_train_X.shape[0] == len(kNN_train_Y)

Validation dataset to choose k

In [192]:
def data_prepare(inf_pid_paths,inf_emb_api,inf_emb,encode_fam=encode_fam,loner_dir=False):
    em_up = 0
    no_ind_rep = 0
    inf_X = []
    inf_Y = []
    for path,api,emb in tqdm(zip(inf_pid_paths,inf_emb_api,inf_emb)):
        if loner_dir:
            fam_name = path.split('/')[-3]
        else:
            fam_name = path.split('/')[-3].split('_')[0].split('.')[-1]
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if length < 1:
            print('Empty profile:',path)
            continue
        try:
            assert api[:length][-1] != 0 #確認profile跟所load 近來的vector是一致的
            assert api[:length+1][-1] == 0
        except:
            print('===Assertion ERR:',i,path)
            continue
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0)
        final_emb_e = emb_model.predict([api_e,emb_e]) #(1,length,768)
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e])
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr)
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if (sum(emb) <0) or (sum(emb)>0):
                rep_emb.append(emb)
        rep_emb_avg = np.average(rep_emb,axis=0)
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):
#                 no_ind_rep_hash.append(recent_hash)
                continue #沒有individual REP
        except ValueError: #正常profile
            pass
        inf_X.append(rep_emb_avg)
        for j,v in enumerate(list(encode_fam.keys())):
            if fam_name in v:
                inf_Y.append(j)
                break        
    assert len(inf_X) == len(inf_Y)
    return np.array(inf_X),inf_Y

In [184]:
valid_X , valid_Y = data_prepare(valid_pid_paths,valid_emb_api,valid_emb)


0it [00:00, ?it/s]
1it [00:00,  4.83it/s]
2it [00:00,  5.17it/s]
3it [00:00,  5.36it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

4it [00:00,  5.34it/s]
5it [00:00,  5.50it/s]
6it [00:01,  5.62it/s]
7it [00:01,  5.47it/s]
8it [00:01,  5.60it/s]
9it [00:01,  5.77it/s]
10it [00:01,  5.98it/s]
11it [00:01,  5.97it/s]
12it [00:02,  5.98it/s]
13it [00:02,  5.84it/s]
14it [00:02,  5.81it/s]
15it [00:02,  5.91it/s]
16it [00:02,  6.00it/s]
17it [00:02,  5.92it/s]
18it [00:03,  6.02it/s]
19it [00:03,  5.97it/s]
20it [00:03,  6.11it/s]
21it [00:03,  6.04it/s]
22it [00:03,  5.95it/s]
23it [00:03,  6.22it/s]
24it [00:04,  5.94it/s]
25it [00:04,  5.81it/s]
26it [00:04,  5.80it/s]
27it [00:04,  5.87it/s]
28it [00:04,  5.93it/s]
29i

285it [00:47,  6.15it/s]
286it [00:47,  6.12it/s]
287it [00:47,  6.24it/s]
288it [00:47,  6.20it/s]
289it [00:47,  6.19it/s]
290it [00:48,  6.32it/s]
291it [00:48,  6.28it/s]
292it [00:48,  6.35it/s]
293it [00:48,  6.26it/s]
294it [00:48,  6.29it/s]
295it [00:48,  6.07it/s]
296it [00:49,  6.40it/s]
297it [00:49,  6.30it/s]
298it [00:49,  6.11it/s]
299it [00:49,  6.03it/s]
300it [00:49,  6.05it/s]
301it [00:49,  6.00it/s]
302it [00:50,  6.03it/s]
303it [00:50,  6.07it/s]
304it [00:50,  5.85it/s]
305it [00:50,  5.95it/s]
306it [00:50,  5.81it/s]
307it [00:50,  5.98it/s]
308it [00:51,  5.97it/s]
309it [00:51,  6.23it/s]
310it [00:51,  6.21it/s]
311it [00:51,  6.26it/s]
312it [00:51,  6.15it/s]
313it [00:51,  5.93it/s]
314it [00:52,  5.76it/s]
315it [00:52,  5.79it/s]
316it [00:52,  5.78it/s]
317it [00:52,  5.77it/s]
318it [00:52,  5.76it/s]
319it [00:52,  5.76it/s]
320it [00:53,  5.75it/s]
321it [00:53,  5.78it/s]
322it [00:53,  5.86it/s]
323it [00:53,  5.81it/s]
324it [00:53,  5.74it/s]


In [191]:
for i in [3,5,7,9,11,13,15,17,19,21,23,25,27,29,31]:
    neigh = KNeighborsClassifier(n_neighbors=i,n_jobs=30,weights='distance') #,weights='distance'
    neigh.fit(kNN_train_X, kNN_train_Y) 
    score = neigh.score(valid_X,valid_Y)
    print('k=',i,',acc=',score)

k= 3 ,acc= 0.7404129793510325
k= 5 ,acc= 0.7522123893805309
k= 7 ,acc= 0.7522123893805309
k= 9 ,acc= 0.7610619469026548
k= 11 ,acc= 0.7551622418879056
k= 13 ,acc= 0.7581120943952803
k= 15 ,acc= 0.7551622418879056
k= 17 ,acc= 0.7551622418879056
k= 19 ,acc= 0.7551622418879056
k= 21 ,acc= 0.7581120943952803
k= 23 ,acc= 0.7581120943952803
k= 25 ,acc= 0.7581120943952803
k= 27 ,acc= 0.7581120943952803
k= 29 ,acc= 0.7581120943952803
k= 31 ,acc= 0.7581120943952803


In [193]:
exp_X , exp_Y = data_prepare(exp_pid_paths,exp_api,exp_emb)


0it [00:00, ?it/s]
1it [00:00,  5.89it/s]
2it [00:00,  6.08it/s]
3it [00:00,  6.25it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

4it [00:00,  6.56it/s]
5it [00:00,  6.54it/s]
6it [00:00,  6.44it/s]
7it [00:01,  6.32it/s]
8it [00:01,  6.03it/s]
9it [00:01,  6.01it/s]
11it [00:01,  7.24it/s]

Empty profile: ./data/tree-rep-profiles_o2o/EXP/parite/G0/bab4f83751a14be7ba6d757674024de4a2c043556a51039eb4708e8b5de0683d_2892.profile
Empty profile: ./data/tree-rep-profiles_o2o/EXP/parite/G0/67df737fad5c83781f20c9485749ef999f88e012c1ecdadabf1c8ab3b8f14a00_432.profile



13it [00:01,  8.15it/s]
15it [00:01,  8.71it/s]

Empty profile: ./data/tree-rep-profiles_o2o/EXP/parite/G0/cc17c0d303ea565d667001978d92b92e6866fbf156aea8a048e8c3b3134904f8_3336.profile



16it [00:02,  7.97it/s]
17it [00:02,  7.36it/s]
18it [00:02,  7.12it/s]
19it [00:02,  6.86it/s]
20it [00:02,  6.56it/s]
21it [00:02,  6.51it/s]
22it [00:03,  6.50it/s]
23it [00:03,  6.30it/s]
24it [00:03,  6.20it/s]
25it [00:03,  6.12it/s]
26it [00:03,  5.89it/s]
27it [00:03,  6.02it/s]
28it [00:04,  5.94it/s]
29it [00:04,  6.11it/s]
30it [00:04,  6.08it/s]
31it [00:04,  5.97it/s]
32it [00:04,  6.16it/s]
33it [00:04,  6.22it/s]
34it [00:05,  5.98it/s]
35it [00:05,  6.17it/s]
36it [00:05,  5.93it/s]
37it [00:05,  5.79it/s]
38it [00:05,  5.83it/s]
39it [00:05,  5.74it/s]
41it [00:06,  6.76it/s]

...


3808it [09:28,  7.18it/s]
3809it [09:28,  6.70it/s]
3810it [09:28,  6.63it/s]
3811it [09:28,  6.07it/s]
3812it [09:28,  6.24it/s]
3813it [09:28,  6.27it/s]
3814it [09:29,  6.40it/s]
3815it [09:29,  6.39it/s]
3816it [09:29,  6.44it/s]
3817it [09:29,  6.50it/s]
3819it [09:29,  7.69it/s]

Empty profile: ./data/tree-rep-profiles_o2o/EXP/virut/G0/affe911078c6475886ea493474aa60e462efe3a34021096222eeb7f5e1730544_3432.profile



3820it [09:29,  7.28it/s]
3821it [09:29,  6.46it/s]
3822it [09:30,  6.48it/s]
3823it [09:30,  6.24it/s]
3824it [09:30,  6.03it/s]
3825it [09:30,  5.97it/s]
3826it [09:30,  5.89it/s]
3827it [09:31,  5.75it/s]
3828it [09:31,  5.96it/s]
3829it [09:31,  5.68it/s]
3830it [09:31,  6.02it/s]
3831it [09:31,  5.83it/s]
3832it [09:31,  5.89it/s]
3833it [09:32,  5.90it/s]
3834it [09:32,  5.71it/s]
3835it [09:32,  5.89it/s]
3836it [09:32,  5.76it/s]
3837it [09:32,  5.96it/s]
3838it [09:32,  5.63it/s]
3839it [09:33,  5.71it/s]
3841it [09:33,  6.78it/s]

Empty profile: ./data/tree-rep-profiles_o2o/EXP/virut/G0/26f99e9e43486ad354c363c7307062ea74d829aa6face0060475848f14c7e978_3216.profile



3842it [09:33,  6.43it/s]
3843it [09:33,  6.42it/s]
3844it [09:33,  6.28it/s]
3845it [09:33,  6.10it/s]
3846it [09:34,  6.07it/s]
3847it [09:34,  6.29it/s]
3848it [09:34,  5.96it/s]
3849it [09:34,  6.16it/s]
3850it [09:34,  6.17it/s]
3851it [09:34,  6.08it/s]
3852it [09:35,  6.15it/s]
3853it [09:35,  6.19it/s]
3854it [09:35,  6.14it/s]
3855it [09:35,  6.01it/s]
3856it [09:35,  6.20it/s]
3857it [09:35,  6.19it/s]
3858it [09:36,  6.26it/s]
3859it [09:36,  6.32it/s]
3860it [09:36,  6.38it/s]
3861it [09:36,  6.54it/s]
3862it [09:36,  6.07it/s]
3863it [09:36,  6.16it/s]
3864it [09:36,  6.21it/s]
3865it [09:37,  6.23it/s]
3866it [09:37,  6.23it/s]
3867it [09:37,  6.38it/s]
3868it [09:37,  6.02it/s]
3869it [09:37,  6.20it/s]
3870it [09:37,  6.18it/s]
3871it [09:38,  6.15it/s]
3872it [09:38,  6.02it/s]
3873it [09:38,  6.15it/s]
3874it [09:38,  6.15it/s]
3875it [09:38,  6.28it/s]
3876it [09:38,  5.84it/s]
3877it [09:39,  6.00it/s]
3878it [09:39,  5.70it/s]
3879it [09:39,  6.16it/s]
3880it [09:

In [200]:
neigh = KNeighborsClassifier(n_neighbors=1311,n_jobs=30) #,weights='distance'
neigh.fit(kNN_train_X, kNN_train_Y) 
score = neigh.score(exp_X,exp_Y)
ans = neigh.predict(exp_X)
score

0.06925465838509316

In [198]:
ans

array([43, 30, 30, ..., 30, 39,  9])

# Deprecated

## Train
* cluster each tree
* avg vector
* threashold
* 改成跟每個call比較距離?
* 改成cosine sim?

In [168]:
fam_name = None
tree_name = None
tree_vec = []
root_dir = './data/tree-rep-profiles_o2o/centroids/'
if not os.path.exists(root_dir):
    os.makedirs(root_dir,exist_ok=True)
for path,api,emb in tqdm(zip(train_pid_paths,train_emb_api,train_emb)):
    recent_fam = path.split('/')[-3].split('_')[0]
    recent_tree = path.split('/')[-2]
    with open(path,encoding='ISO 8859-1') as f: #X2
        lines = f.read()
    lines = re.sub(r'[^\x00-\x7F]+','', lines)
    lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
    lines = re.sub('=\\n','',lines)
    lines = re.sub('y\\n','',lines)
    lines = lines.splitlines()
    length = len(lines)
    if length < 11:
        print('No common rep. Should not in a tree:',path)
        continue
#     print(len(lines))
    assert api[:length][-1] != 0
    assert api[:length+1][-1] == 0
    api_e = np.expand_dims(api, axis=0)
    emb_e = np.expand_dims(emb, axis=0)
    final_emb_e = emb_model.predict([api_e,emb_e]) #(1,length,768)
    final_emb = final_emb_e[0][:length] #(length,768)
    byte_rep_e = model.predict([api_e,emb_e])
    byte_rep = byte_rep_e[0][:length] # (length,1)
    byte_rep_thr = []
    for num in byte_rep:
        if num[0] < thr:
            byte_rep_thr.append(0)
        else:
            byte_rep_thr.append(1) # high than threshold, keep it
    byte_rep_thr = np.array(byte_rep_thr)
    byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
    mul_emb = np.multiply(final_emb,byte_rep_thr)
    rep_emb = [] #超過threshold的emb才留下來
    for emb in mul_emb:
        if (sum(emb) <0) or (sum(emb)>0):
            rep_emb.append(emb)
    rep_emb_avg = np.average(rep_emb,axis=0) #把自己各api的embedding取平均，應該一定要有，不然就代表這個profile不該屬於這棵tree
    if recent_fam==fam_name and recent_tree==tree_name:
        tree_vec.append(rep_emb_avg) #如果同tree同fam就把rep_emb加進去list
    else:
        if tree_vec:
            fam_tree_c = np.average(tree_vec,axis=0) # 求出整個tree的centroid
            pickle.dump(file=open(root_dir+fam_name+'_'+tree_name+'.pkl','wb'),obj=fam_tree_c)
            tree_vec = []
        fam_name = recent_fam
        tree_name = recent_tree
        if not tree_vec: # if tree vector list is empty
            tree_vec.append(rep_emb_avg)
#     break
#     print(path,api,emb)
#     break

114it [00:17,  7.03it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
1538it [03:46,  7.47it/s]

No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/normal/125.krypt_0.8/G24/ef944eb8743d249656d79d648a88e31f54a4cc3c10dacc86f33a9f5eb4100de5_3372.profile


1545it [03:47, 10.52it/s]

No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/normal/125.krypt_0.8/G24/1cd7a58cb78d893a0c943547afd917fd446cdc16a9259286062c7972dadeba77_3300.profile
No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/normal/125.krypt_0.8/G24/12306d8f978ba2830dc207d05c9b1963edacca2d10a935654cbdcecc3e744b7b_3312.profile
No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/normal/125.krypt_0.8/G24/ef944eb8743d249656d79d648a88e31f54a4cc3c10dacc86f33a9f5eb4100de5_3620.profile
No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/normal/125.krypt_0.8/G24/ef944eb8743d249656d79d648a88e31f54a4cc3c10dacc86f33a9f5eb4100de5_3276.profile
No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/normal/125.krypt_0.8/G24/ef944eb8743d249656d79d648a88e31f54a4cc3c10dacc86f33a9f5eb4100de5_3412.profile
No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/normal/125.krypt_0.8/G24/ef944eb8743d249656d79d648a88e31f54a4cc3c10dacc86f33a9f5eb4100de5

1937it [04:44,  8.17it/s]

...

9750it [22:34,  8.02it/s]

No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/normal/21.soft_0.8/G222/4c6cbd4288806a30edc643a4d4bdbbc80d6a452680ed74f4be2ac2ff6a2d063a_3352.profile


9765it [22:36,  8.20it/s]

No common rep. Should not in a tree: ./data/tree-rep-profiles_o2o/normal/21.soft_0.8/G222/0faf56b2177f5b411e4a8dd077634b0988e32b54bb47b0da7e3de0cd86412df7_3220.profile


11455it [26:48,  5.94it/s]


## Dev / Test
* calculate euclidean distance
* check if match right tree/fam
* top1/top3/top5
* 同sample不同pid的歸類，同分的歸類 (?)
* topN dis<0.2以下的歸類 (?)
* family改成是最近的3/5個family (同family就往下推移)
* 把training沒有此tree的DEV pid拿掉

In [257]:
centroid_dir = './data/tree-rep-profiles_o2o/centroids/' #要有最後的反斜線
def inf_evaluate(inf_pid_paths,inf_emb_api,inf_emb,centroid_dir=centroid_dir):
    centroids = next(os.walk(centroid_dir))[2]
    centroid_matrix = None
    c_fam = []
    c_tree = []
    err_centroid = []
    top1_fam_up = 0
    top1_tree_up = 0
    top3_fam_up = 0
    top3_tree_up = 0
    top5_fam_up = 0
    top5_tree_up = 0
    wrong_fam = {}
    wrong_tree = {}
    no_ind_rep = 0
    for i,c in enumerate(centroids):
        tree_rep = pickle.load(open(centroid_dir + c,'rb')) #每個tree的centroid pickle
        tree_rep = np.expand_dims(tree_rep,axis=0)
        try:
            chk = sum(sum(tree_rep))
        except TypeError:
            err_centroid.append(c)
            continue
        if (not chk < 0) and (not chk>0):
            err_centroid.append(c)
            continue
        if i >0:
            centroid_matrix = np.concatenate((centroid_matrix,tree_rep),axis=0) #依順序合在一起       
        if i==0:
            centroid_matrix = tree_rep # 1st time
        c_fam.append(c.split('_')[0]) # centroid的fam name
        c_tree.append(c.split('_')[-1].split('.')[0]) # centroid的tree name
    assert centroid_matrix.shape[0] == len(c_fam) == len(c_tree)
    assert len(inf_pid_paths) == len(inf_emb_api) == len(inf_emb)
    for path,api,emb in tqdm(zip(inf_pid_paths,inf_emb_api,inf_emb)):
        recent_fam = path.split('/')[-3].split('_')[0]
        recent_tree = path.split('/')[-2]
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if length < 1:
            print('Empty profile:',path)
            continue
        assert api[:length][-1] != 0 #確認profile跟所load 近來的vector是一致的
        assert api[:length+1][-1] == 0
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0)
        final_emb_e = emb_model.predict([api_e,emb_e])
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e])
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr) #把embediing跟超過threshold的byterep相乘  
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if sum(emb) !=0: #跟0鄉城都會=0
                rep_emb.append(emb)       
        rep_emb_avg = np.average(rep_emb,axis=0)
        rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):
                no_ind_rep+=1
                continue #沒有individual REP
        except ValueError: #正常profile
            pass
        dist = euclidean_distances(rep_emb_avg,centroid_matrix) #取出與各rep距離，求min的idx對答案
        min_dist = np.min(dist) #top1
        min_idx = np.argmin(dist) #top1
        dist5 = dist[0].argsort()[:5] #top3
        if c_fam[min_idx] == recent_fam:
            top1_fam_up = top1_fam_up+1
        else:
            wrong_fam[path] = c_fam[min_idx]
        if (c_fam[min_idx] == recent_fam) and (c_tree[min_idx] == recent_tree):
            top1_tree_up = top1_tree_up+1
        else:
            wrong_tree[path] = c_fam[min_idx]+'_'+ c_tree[min_idx]
        if recent_fam in [c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]]]:
            top3_fam_up = top3_fam_up+1
        if (recent_tree in [c_tree[dist5[0]],c_tree[dist5[1]],c_tree[dist5[2]]]) and (recent_fam in[c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]]]):
            top3_tree_up = top3_tree_up+1
        if recent_fam in [c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]],c_fam[dist5[3]],c_fam[dist5[4]]]:
            top5_fam_up = top5_fam_up+1
        if (recent_fam in [c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]],c_fam[dist5[3]],c_fam[dist5[4]]]) and (recent_tree in [c_tree[dist5[0]],c_tree[dist5[1]],c_tree[dist5[2]],c_tree[dist5[3]],c_tree[dist5[4]]]):
            top5_tree_up = top5_tree_up + 1
    down = len(inf_pid_paths)
    print('Top1 Fam score:',top1_fam_up/down,'Top1 Tree score:',top1_tree_up/down,
         'Top3 Fam score:',top3_fam_up/down,'Top3 Tree score:',top3_tree_up/down,
          'Top5 Fam score:',top5_fam_up/down,'Top5 Tree score:', top5_tree_up/down,
          'No ind rep#:',no_ind_rep
         )
    return wrong_fam,wrong_tree
#         for c in centroids:
#             tree_rep = pickle.load(open(centroid_dir + c,'rb'))


In [258]:
temp = inf_evaluate(test_pid_paths,test_emb_api,test_emb)


0it [00:00, ?it/s]
1it [00:00,  6.89it/s]
2it [00:00,  6.92it/s]
3it [00:00,  6.19it/s]
5it [00:00,  6.89it/s]
6it [00:00,  7.23it/s]
7it [00:00,  7.36it/s]
8it [00:01,  7.51it/s]
9it [00:01,  7.71it/s]
10it [00:01,  7.70it/s]
11it [00:01,  8.07it/s]
12it [00:01,  8.40it/s]
13it [00:01,  7.97it/s]
14it [00:01,  8.01it/s]
15it [00:01,  8.14it/s]
16it [00:02,  7.88it/s]
17it [00:02,  7.73it/s]
18it [00:02,  7.58it/s]
19it [00:02,  7.63it/s]
20it [00:02,  7.49it/s]
21it [00:02,  7.59it/s]
22it [00:02,  7.78it/s]
23it [00:02,  8.04it/s]
24it [00:03,  8.20it/s]
25it [00:03,  8.26it/s]
26it [00:03,  8.43it/s]
27it [00:03,  8.36it/s]
28it [00:03,  8.20it/s]
29it [00:03,  7.97it/s]
30it [00:03,  8.00it/s]
31it [00:03,  7.99it/s]
32it [00:04,  7.92it/s]
33it [00:04,  7.95it/s]
34it [00:04,  7.94it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/c

Empty profile: ./data/tree-rep-profiles_o2o/TEST/27.downloa_0.8/G184/0f3dbceacbbebc3dd86d6f7c24f964f8c20a46d40c265701cb2ce2b557858188_3284.profile


/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

166it [00:20,  9.31it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

168it [00:21,  9.07it/s]
169it [00:21,  8.69it/s]
170it [00:21,  8.33it/s]
171it [00:21,  8.24it/s]
172it [00:21,  7.91it/s]
173it [00:21,  7.76it/s]
174it [00:21,  7.79it/s]
175it [00:22,  7.78it/s]
176it [00:22,  7.78it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  a

Empty profile: ./data/tree-rep-profiles_o2o/TEST/27.downloa_0.8/G208/0f3dbceacbbebc3dd86d6f7c24f964f8c20a46d40c265701cb2ce2b557858188_3284.profile



225it [00:28,  9.22it/s]
226it [00:28,  8.70it/s]
228it [00:28,  9.92it/s]

...


655it [01:20,  8.88it/s]
656it [01:20,  8.46it/s]
657it [01:20,  8.35it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

658it [01:20,  8.49it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

660it [01:21,  8.81it/s]
661it [01:21,  8.93it/s]
662it [01:21,  8.85it/s]
663it [01:21,  8.70it/s]
664it [01:21,  8.16it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/l

755it [01:32,  8.13it/s]
757it [01:32,  8.28it/s]
758it [01:33,  7.94it/s]
759it [01:33,  8.27it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

760it [01:33,  8.28it/s]
761it [01:33,  8.40it/s]
762it [01:33,  8.03it/s]
763it [01:33,  8.16it/s]
764it [01:33,  8.26it/s]
765it [01:33,  7.62it/s]
766it [01:34,  7.70it/s]
767it [01:34,  8.00it/s]
768it [01:34,  7.86it/s]
769it [01:34,  8.08it/s]
770it [01:34,  8.04it/s]
771it [01:34,  8.06it/s]
772it [01:34,  8.09it/s]
773it [01:34,  7.66it/s]
774it [01:35,  7.49it/s]
775it [01:35,  7.66it/s]
776it [01:35,  7.70it/s]
778it [01:35,  8.04it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axi

Empty profile: ./data/tree-rep-profiles_o2o/TEST/21.soft_0.8/G199/4c6cbd4288806a30edc643a4d4bdbbc80d6a452680ed74f4be2ac2ff6a2d063a_3508.profile



851it [01:44,  9.62it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

853it [01:44,  9.63it/s]
855it [01:44,  8.99it/s]
856it [01:44,  8.92it/s]
857it [01:44,  8.74it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

858it [01:44,  8.71it/s]
860it [01:45,  8.53it/s]
861it [01:45,  8.52it/s]
862it [01:45,  8.69it/s]
863it [01:45,  8.57it/s]
864it [01:45,  8.37it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning:

Top1 Fam score: 0.060114503816793896 Top1 Tree score: 0.04484732824427481 Top3 Fam score: 0.14026717557251908 Top3 Tree score: 0.10019083969465649 Top5 Fam score: 0.18129770992366412 Top5 Tree score: 0.1364503816793893 No ind rep#: 95


## Loner EXP classification

In [269]:
centroid_dir = './data/tree-rep-profiles_o2o/centroids/' #要有最後的反斜線
def loner_evaluate(lon_pid_paths,lon_emb_api,lon_emb,centroid_dir=centroid_dir):
    centroids = next(os.walk(centroid_dir))[2]
    centroid_matrix = None
    c_fam = []
    c_tree = []
    err_centroid = []
    top1_fam_up = 0
#     top1_tree_up = 0
    top3_fam_up = 0
#     top3_tree_up = 0
    top5_fam_up = 0
#     top5_tree_up = 0
    wrong_fam = {}
#     wrong_tree = {}
    no_ind_rep = 0
    for i,c in enumerate(centroids):
        tree_rep = pickle.load(open(centroid_dir + c,'rb')) #每個tree的centroid pickle
        tree_rep = np.expand_dims(tree_rep,axis=0)
        try:
            chk = sum(sum(tree_rep))
        except TypeError:
            err_centroid.append(c)
            continue
        if (not chk < 0) and (not chk>0):
            err_centroid.append(c)
            continue
        if i >0:
            centroid_matrix = np.concatenate((centroid_matrix,tree_rep),axis=0) #依順序合在一起       
        if i==0:
            centroid_matrix = tree_rep # 1st time
        c_fam.append(c.split('_')[0]) # centroid的fam name
#         c_tree.append(c.split('_')[-1].split('.')[0]) # centroid的tree name
    assert centroid_matrix.shape[0] == len(c_fam) #== len(c_tree)
    assert len(lon_pid_paths) == len(lon_emb_api) == len(lon_emb)
    for path,api,emb in tqdm(zip(lon_pid_paths,lon_emb_api,lon_emb)):
        recent_fam = path.split('/')[-3].split('_')[0]
#         recent_tree = path.split('/')[-2]
        with open(path,encoding='ISO 8859-1') as f: #X2
            lines = f.read()
        lines = re.sub(r'[^\x00-\x7F]+','', lines)
        lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
        lines = re.sub('=\\n','',lines)
        lines = re.sub('y\\n','',lines)
        lines = lines.splitlines()
        length = len(lines)
        if length < 1:
            print('Empty profile:',path)
            continue
        assert api[:length][-1] != 0 #確認profile跟所load 近來的vector是一致的
        assert api[:length+1][-1] == 0
        api_e = np.expand_dims(api, axis=0)
        emb_e = np.expand_dims(emb, axis=0)
        final_emb_e = emb_model.predict([api_e,emb_e])
        final_emb = final_emb_e[0][:length] #(length,768)
        byte_rep_e = model.predict([api_e,emb_e])
        byte_rep = byte_rep_e[0][:length] # (length,1)
        byte_rep_thr = []
        for num in byte_rep:
            if num[0] < thr:
                byte_rep_thr.append(0)
            else:
                byte_rep_thr.append(1) # high than threshold, keep it
        byte_rep_thr = np.array(byte_rep_thr)
        byte_rep_thr = np.expand_dims(byte_rep_thr,axis=-1)
        mul_emb = np.multiply(final_emb,byte_rep_thr) #把embediing跟超過threshold的byterep相乘  
        rep_emb = [] #超過threshold的emb才留下來
        for emb in mul_emb:
            if sum(emb) !=0: #跟0鄉城都會=0
                rep_emb.append(emb)       
        rep_emb_avg = np.average(rep_emb,axis=0)
        rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)
        try:
            if not (rep_emb_avg <0 or rep_emb_avg>0):#沒有individual REP
                no_ind_rep+=1
                continue 
        except ValueError: #正常profile
            pass
        dist = euclidean_distances(rep_emb_avg,centroid_matrix) #取出與各rep距離，求min的idx對答案
        min_dist = np.min(dist) #top1
        min_idx = np.argmin(dist) #top1
        dist5 = dist[0].argsort()[:5] #top3
        if c_fam[min_idx] == recent_fam:
            top1_fam_up = top1_fam_up+1
        else:
            wrong_fam[path] = c_fam[min_idx]
#         if (c_fam[min_idx] == recent_fam) and (c_tree[min_idx] == recent_tree):
#             top1_tree_up = top1_tree_up+1
#         else:
#             wrong_tree[path] = c_fam[min_idx]+'_'+ c_tree[min_idx]
        if recent_fam in [c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]]]:
            top3_fam_up = top3_fam_up+1
#         if (recent_tree in [c_tree[dist5[0]],c_tree[dist5[1]],c_tree[dist5[2]]]) and (recent_fam in[c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]]]):
#             top3_tree_up = top3_tree_up+1
        if recent_fam in [c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]],c_fam[dist5[3]],c_fam[dist5[4]]]:
            top5_fam_up = top5_fam_up+1
#         if (recent_fam in [c_fam[dist5[0]],c_fam[dist5[1]],c_fam[dist5[2]],c_fam[dist5[3]],c_fam[dist5[4]]]) and (recent_tree in [c_tree[dist5[0]],c_tree[dist5[1]],c_tree[dist5[2]],c_tree[dist5[3]],c_tree[dist5[4]]]):
#             top5_tree_up = top5_tree_up + 1
    down = len(lon_pid_paths)
    print('Top1 Fam score:',top1_fam_up/down,#'Top1 Tree score:',top1_tree_up/down,
         'Top3 Fam score:',top3_fam_up/down,#'Top3 Tree score:',top3_tree_up/down,
          'Top5 Fam score:',top5_fam_up/down,#'Top5 Tree score:', top5_tree_up/down,
          'No ind rep#:',no_ind_rep
         )
    return wrong_fam
#         for c in centroids:
#             tree_rep = pickle.load(open(centroid_dir + c,'rb'))


In [270]:
temp = loner_evaluate(exp_pid_paths,exp_api,exp_emb)


0it [00:00, ?it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

1it [00:00,  6.42it/s]
2it [00:00,  7.10it/s]
3it [00:00,  7.50it/s]
4it [00:00,  7.71it/s]
5it [00:00,  7.90it/s]
6it [00:00,  7.91it/s]
7it [00:00,  8.06it/s]
8it [00:00,  8.24it/s]
9it [00:01,  7.99it/s]
10it [00:01,  8.02it/s]
11it [00:01,  8.05it/s]
12it [00:01,  8.12it/s]
13it [00:01,  8.15it/s]
14it [00:01,  8.01it/s]
15it [00:01,  8.01it/s]
16it [00:01,  7.99it/s]
17it [00:02,  7.95it/s]
18it [00:02,  7.75it/s]
19it [00:02,  7.89it/s]
20it [00:02,  8.01it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/

124it [00:15,  8.39it/s]
125it [00:15,  8.46it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

126it [00:15,  8.55it/s]
127it [00:15,  8.71it/s]
128it [00:15,  8.42it/s]
129it [00:15,  8.27it/s]
130it [00:15,  7.94it/s]
131it [00:15,  7.71it/s]
132it [00:16,  7.97it/s]
133it [00:16,  8.19it/s]
134it [00:16,  8.27it/s]
135it [00:16,  8.26it/s]
136it [00:16,  8.31it/s]
137it [00:16,  8.29it/s]
138it [00:16,  8.20it/s]
139it [00:16,  8.13it/s]
140it [00:17,  8.11it/s]
141it [00:17,  7.88it/s]
142it [00:17,  7.72it/s]
143it [00:17,  7.68it/s]
144it [00:17,  7.27it/s]
145it [00:17,  6.88it/s]
146it [00:17,  6.91it/s]
147it [00:18,  6.93it/s]
148it [00:18,  7.03it/s]
149it [00:18,  6.95it/s]
150it [00:18,  7.05it/s]
151it [00:1

291it [00:36,  8.14it/s]
292it [00:36,  8.56it/s]
293it [00:36,  8.55it/s]
294it [00:36,  8.37it/s]/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/leoqaz12/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

295it [00:36,  8.52it/s]
297it [00:36,  8.63it/s]
298it [00:36,  8.43it/s]
299it [00:37,  8.31it/s]
300it [00:37,  8.32it/s]
301it [00:37,  8.11it/s]
302it [00:37,  7.74it/s]
303it [00:37,  7.40it/s]
304it [00:37,  7.58it/s]
305it [00:37,  7.42it/s]
306it [00:37,  7.42it/s]
307it [00:38,  7.52it/s]
308it [00:38,  7.90it/s]
309it [00:38,  7.97it/s]
310it [00:38,  8.00it/s]
311it [00:38,  8.09it/s]
312it [00:38,  8.11it/s]
313it [00:38,  8.27it/s]
314it [00:38,  8.04it/s]
315it [00:39,  8.03it/s]
316it [00:39,  8.14it/s]
317it [00:39,  8.20it/s]
318it [00:39,  8.13it/s]
319it [00:3

Top1 Fam score: 0.17045454545454544 Top3 Fam score: 0.29545454545454547 Top5 Fam score: 0.34375 No ind rep#: 37


NameError: name 'wrong_tree' is not defined

In [264]:
centroids[0].split('_')[0]

'2.zbot'

In [236]:
dist_s = dist[0].argsort()[:3]

In [238]:
dist_s[0]

118

In [228]:
np.argmin(dist)

118

In [227]:
dist[0][118]

2.3757918363134127

In [214]:
rep_emb_avg = np.expand_dims(rep_emb_avg,axis=0)

In [210]:
i

830

In [188]:
tree_rep = pickle.load(open(centroid_dir + '20.ramnit_G420.pkl','rb'))
tree_rep

nan

In [218]:
centroid_matrix.shape

(729, 768)

In [230]:
recent_tree

'G22'

In [164]:
tree_rep = pickle.load(open(root_dir + '22.razy_0.8_G281.pkl','rb'))
kkkk = np.expand_dims(tree_rep,axis=0)
# tree_rep = 
kkkk

array([[ 8.65596963e-02, -1.38961762e+00, -8.80325557e-01,
         1.44868998e+00,  3.33986451e+00,  1.84224879e+00,
        -1.22117380e+00, -1.73912917e+00, -2.14322432e+00,
        -2.42802131e+00,  7.90877797e-01, -9.03681114e-01,
        -1.33191333e+00, -1.42146935e+00, -3.65049951e+00,
        -3.09185951e-01,  2.77627880e+00,  3.25421317e+00,
         1.27467083e+00, -2.09883721e-01,  1.37831007e+00,
        -1.50192061e+00,  1.86882590e-01,  2.07811224e+00,
        -3.61454106e+00, -2.11806000e+00,  1.17014932e-01,
        -9.56569921e-01, -3.29551819e-01, -7.14762268e-01,
        -6.92955113e-01,  2.05578955e+00,  2.02854938e+00,
         1.70297706e+00, -1.78908776e+00,  1.28335643e+00,
        -5.62766004e+00,  2.14058878e-01,  6.18024734e-02,
         2.52637649e+00,  2.05098724e-01,  3.90071236e-01,
         4.08042537e-01, -2.85100074e-01,  1.17381613e+00,
         3.74853606e+00, -1.58441165e+00,  9.09052398e-01,
         2.28420114e+00,  2.40741630e+00, -5.56467452e-0

In [169]:
k = np.expand_dims(rep_emb_avg,axis=0)
kk = np.repeat(k,5,axis=0)
k

array([[-2.32679514e-03, -1.51152636e+00, -1.32905939e+00,
         9.56873546e-01,  3.48319108e+00,  1.81062158e+00,
        -1.19937576e+00, -1.57059526e+00, -2.14856078e+00,
        -2.66518752e+00,  2.08959677e-01, -1.13687739e+00,
        -1.03918429e+00, -6.80092806e-01, -3.51498023e+00,
        -2.18805710e-01,  2.73317995e+00,  3.09727030e+00,
         1.15348836e+00, -1.18641081e+00,  1.43051075e+00,
        -1.39607588e+00,  7.68887893e-01,  2.14773534e+00,
        -3.52791246e+00, -1.77533390e+00, -7.99679553e-02,
        -5.23277587e-01, -7.02902425e-01, -8.52432431e-01,
        -4.29765171e-01,  2.15343165e+00,  1.83647086e+00,
         6.52459159e-01, -1.55454592e+00,  1.03299757e+00,
        -4.88682050e+00,  2.14824384e-02,  4.52365744e-01,
         3.43573372e+00,  3.48641196e-01,  6.86164492e-01,
         4.88668572e-01, -7.38335954e-01,  7.35207275e-01,
         3.97278129e+00, -7.10923588e-01, -1.35453728e-01,
         2.37253270e+00,  2.09358406e+00, -7.30818144e-0

In [135]:
rep_emb_avg.shape

(768,)

In [167]:
k = None
np.concatenate((kkkk,k),axis=0)

ValueError: all the input arrays must have same number of dimensions

In [153]:
euclidean_distances(kk,k)

array([[12.34622314],
       [12.34622314],
       [12.34622314],
       [12.34622314],
       [12.34622314]])

In [131]:
next(os.walk(root_dir))[2]

['22.razy_0.8_G281.pkl',
 '19.installcore_0.8_G344.pkl',
 '139.softonic_0.8_G34.pkl',
 '13.vobfus_0.8_G462.pkl',
 '2.zbot_0.8_G1370.pkl',
 '25.browsefox_0.8_G309.pkl',
 '6.virut_0.8_G1418.pkl',
 '37.somoto_0.8_G215.pkl',
 '4.kazy_0.8_G1242.pkl',
 '9.loadmoney_0.8_G614.pkl',
 '16.sytro_0.8_G485.pkl',
 '29.parite_0.8_G298.pkl',
 '11.zusy_0.8_G556.pkl',
 '52.megasearch_0.8_G132.pkl',
 '25.browsefox_0.8_G304.pkl',
 '5.ursu_0.8_G787.pkl',
 '7.solimba_0.8_G740.pkl',
 '57.vilsel_0.8_G433.pkl',
 '3.allaple_0.8_G1429.pkl',
 '39.sirefef_0.8_G119.pkl',
 '54.fesber_0.8_G351.pkl',
 '10.symmi_0.8_G609.pkl',
 '2.zbot_0.8_G1355.pkl',
 '187.wenper_0.8_G17.pkl',
 '32.outbrowse_0.8_G266.pkl',
 '2.zbot_0.8_G1317.pkl',
 '59.installrex_0.8_G96.pkl',
 '5.ursu_0.8_G999.pkl',
 '26.autoit_0.8_G256.pkl',
 '3.allaple_0.8_G2028.pkl',
 '141.peed_0.8_G19.pkl',
 '21.soft_0.8_G250.pkl',
 '5.ursu_0.8_G1035.pkl',
 '124.mediafinder_0.8_G18.pkl',
 '4.kazy_0.8_G1252.pkl',
 '13.vobfus_0.8_G447.pkl',
 '20.ramnit_0.8_G516.pkl